In [15]:
import pandas as pd
from pathlib import Path
import os
import glob
import re

In [16]:
HOME_DIR = Path.home() / 'Desktop' / 'clir' / 'data' / 'wikidata' / 'base_data'

In [17]:
# --------------------- 过滤 query 的 QID ---------------------
QID_search_results_file = str(HOME_DIR / 'base_train_QID_search_results.csv')
QID_filtered_search_results_file = str(HOME_DIR / 'base_train_QID_filtered_search_results.csv')

def filter_qid(original_file: str, filtered_file: str):

    QID_df = pd.read_csv(original_file, encoding='utf-8')

    # columns_to_keep = ['query', 'search_term', 'id', 'label', 'description']
    columns_to_keep = ['query_id', 'query', 'qid']

    # 只取 columns_to_keep 列数据，然后去除 id 列 为NaN的行数据
    QID_filtered_df = QID_df[columns_to_keep].dropna(subset=['qid'])
    # 去除重复项
    QID_filtered_df.drop_duplicates(keep='first', inplace=True)

    # 保存文件
    QID_filtered_df.to_csv(filtered_file, index=False, encoding='utf-8')

filter_qid(QID_search_results_file, QID_filtered_search_results_file)

In [2]:
import ir_datasets
import pandas as pd
dataset_obj = ir_datasets.load('clirmatrix/kk/bi139-base/zh/train')
queries_df = pd.DataFrame(dataset_obj.queries_iter())
for idx, row in queries_df.iterrows():
    if idx == 31:
        print(row['text'])
        break
    

亚历山大一世 (俄国)


In [4]:
a = {"searchinfo": {"search": "亚历山大一世 (俄国)"},"search": [],"success": 1}

b = a.get("search")

In [12]:
b.extend([1])

In [13]:
len(b) > 0

True

In [14]:
b

[1]